In [1]:
import numpy as np

# Example matrix A (3x3)
A = np.array([[3.0, 1.0, 2.0],
              [2.0, 6.0, 1.0],
              [4.0, 1.0, 8.0]])

# Example right-hand side vector b
b = np.array([1.0, 2.0, 3.0])

# Example cost vector c
c = np.array([4.0, 5.0, 6.0])

def ruiz_scaling(A, b, c, iterations=5):
    # Initialize row and column scaling vectors
    D_r = np.ones(A.shape[0])  # Row scaling factors
    D_c = np.ones(A.shape[1])  # Column scaling factors

    # Perform iterative scaling
    for _ in range(iterations):
        # Row scaling
        row_norms = np.linalg.norm(A, axis=1)  # Norms of each row
        D_r *= row_norms  # Update row scaling factors
        A = A / row_norms[:, np.newaxis]  # Scale rows of A
        b = b / row_norms  # Scale b

        # Column scaling
        col_norms = np.linalg.norm(A, axis=0)  # Norms of each column
        D_c *= col_norms  # Update column scaling factors
        A = A / col_norms  # Scale columns of A
        c = c * col_norms  # Scale c

    # Return the scaled matrix, vectors, and scaling factors
    return A, b, c, D_r, D_c

# Apply Ruiz scaling
A_scaled, b_scaled, c_scaled, D_r, D_c = ruiz_scaling(A.copy(), b.copy(), c.copy())

# Print the scaled results
print("Scaled A:\n", A_scaled)
print("Scaled b:", b_scaled)
print("Scaled c:", c_scaled)
print("Row scaling factors D_r:", D_r)
print("Column scaling factors D_c:", D_c)

# Solve the scaled system A_scaled * x_scaled = b_scaled
x_scaled = np.linalg.solve(A_scaled, b_scaled)
print("Scaled solution x_scaled:", x_scaled)

# Reverse the column scaling to get the solution to the original system
x_original = x_scaled / D_c
print("Original solution x:", x_original)

# solve the original system
x = np.linalg.solve(A, b)
print("Original solution x:", x)


Scaled A:
 [[0.82149394 0.28593841 0.48872893]
 [0.30323279 0.9499195  0.13530103]
 [0.48290536 0.1260641  0.86188031]]
Scaled b: [0.26702159 0.2956919  0.35317224]
Scaled c: [3.90052678 4.66921511 6.55631158]
Row scaling factors D_r: [3.74501549 6.76379704 8.49443884]
Column scaling factors D_c: [0.9751317  0.93384302 1.0927186 ]
Scaled solution x_scaled: [0.02294428 0.25268694 0.35995436]
Original solution x: [0.02352941 0.27058824 0.32941176]
Original solution x: [0.02352941 0.27058824 0.32941176]


In [2]:
import numpy as np
from scipy.optimize import linprog

# Example matrix A (3x3)
A = np.array([[3.0, 1.0, 2.0],
              [2.0, 6.0, 1.0],
              [4.0, 1.0, 8.0]])

# Example right-hand side vector b
b = np.array([1.0, 2.0, 3.0])

# Example cost vector c (we want to minimize c^T x)
c = np.array([4.0, 5.0, 6.0])

def ruiz_scaling(A, b, c, iterations=5):
    # Initialize row and column scaling vectors
    D_r = np.ones(A.shape[0])  # Row scaling factors
    D_c = np.ones(A.shape[1])  # Column scaling factors

    # Perform iterative scaling
    for _ in range(iterations):
        # Row scaling
        row_norms = np.linalg.norm(A, axis=1)  # Norms of each row
        D_r *= row_norms  # Update row scaling factors
        A = A / row_norms[:, np.newaxis]  # Scale rows of A
        b = b / row_norms  # Scale b

        # Column scaling
        col_norms = np.linalg.norm(A, axis=0)  # Norms of each column
        D_c *= col_norms  # Update column scaling factors
        A = A / col_norms  # Scale columns of A
        c = c * col_norms  # Scale c

    # Return the scaled matrix, vectors, and scaling factors
    return A, b, c, D_r, D_c

# Apply Ruiz scaling
A_scaled, b_scaled, c_scaled, D_r, D_c = ruiz_scaling(A.copy(), b.copy(), c.copy())


# Use SciPy's linprog to solve the scaled linear program
# Minimize c_scaled^T x, subject to A_scaled @ x = b_scaled
res = linprog(c_scaled, A_eq=A_scaled, b_eq=b_scaled, method='highs')

# Check if optimization was successful
if res.success:
    # Get the scaled solution
    x_scaled = res.x
    print("Scaled solution x_scaled:", x_scaled)

    # Reverse the column scaling to get the solution to the original problem
    x_original = x_scaled / D_c
    print("Original solution x:", x_original)

    # Check the objective value for the original problem
    original_obj_value = np.dot(c, x_original)
    print("Objective value for the original problem c^T x:", original_obj_value)
else:
    print("Optimization failed:", res.message)


Scaled solution x_scaled: [0.02294428 0.25268694 0.35995436]
Original solution x: [0.02352941 0.27058824 0.32941176]
Objective value for the original problem c^T x: 3.4235294117647057


In [3]:
# solve the original system
res_orig = linprog(c, A_eq=A, b_eq=b, method='highs')
if res_orig.success:
    x_orig = res_orig.x
    print("Original solution x:", x_orig)
    print("Objective value for the original problem c^T x:", res_orig.fun)

Original solution x: [0.02352941 0.27058824 0.32941176]
Objective value for the original problem c^T x: 3.423529411764706
